# PH1978L Final Group Project
Authors: Chiu-Feng Yap, Allison Shay, Joy Yoo

Given a dataset of demographic, lifestyle, social and school related attributes of students, **predict student performance (G3)- (final grade; numeric from 0 to 20)**.

**Three scenarios of predictions will be considered:**
* 1- Classification with two levels (pass/fail)
* 2- Classification with five levels (from I - excellent to V - insufficient)
* 3- Regression, with a numeric output that ranges between 0 and 20

Should include comparison between different machine-learning models (one of which must be linear, and at least 2 non-linear models).

Consider different scenarios where we exclude G1 and G2 variables from our models.

In [39]:
# import libraries we will be using:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import mean
from numpy import absolute
from numpy import loadtxt
import seaborn as sns
from scipy.stats import chi2_contingency, boxcox
%matplotlib inline
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn import metrics, preprocessing
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.neighbors import KNeighborsClassifier

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import warnings 
warnings.filterwarnings(action= 'ignore')

In [2]:
# read in the data
# df = pd.read_csv("./Data/school_grades_dataset.csv")
# df = pd.read_csv("/Users/joy4031/OneDrive - The University of Texas Health Science Center at Houston/F21/1978/school_grades_dataset.csv")
df = pd.read_csv("/Users/hunter/Downloads/school_grades_dataset.csv")

# Scenario 1 -  Classification with two levels (pass/fail);

In [3]:
df1 = df

In [4]:
# create two level categorical variable for classification with two levels (pass/fail)
# create a list of our conditions
conditions = [
    (df['G3'] <= 10),
    (df['G3'] > 10)
    ]

# create a list of the values we want to assign for each condition
values = ['fail', 'pass' ]

# create a new column and use np.select to assign values to it using our lists as arguments
df1['G3_pass_fail'] = np.select(conditions, values)

In [5]:
df1.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pass_fail
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11,pass
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11,pass
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12,pass
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14,pass
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13,pass
5,GP,M,16,U,LE3,T,4,3,services,other,reputation,mother,1,2,0,no,yes,no,yes,yes,yes,yes,no,5,4,2,1,2,5,6,12,12,13,pass
6,GP,M,16,U,LE3,T,2,2,other,other,home,mother,1,2,0,no,no,no,no,yes,yes,yes,no,4,4,4,1,1,3,0,13,12,13,pass
7,GP,F,17,U,GT3,A,4,4,other,teacher,home,mother,2,2,0,yes,yes,no,no,yes,yes,no,no,4,1,4,1,1,1,2,10,13,13,pass
8,GP,M,15,U,LE3,A,3,2,services,other,home,mother,1,2,0,no,yes,no,no,yes,yes,yes,no,4,2,2,1,1,1,0,15,16,17,pass
9,GP,M,15,U,GT3,T,3,4,other,other,home,mother,1,2,0,no,yes,no,yes,yes,yes,yes,no,5,5,1,1,1,5,0,12,12,13,pass


In [ ]:
df1.describe()

In [ ]:
df1.dtypes

In [6]:
X = df1.drop(['G3', 'G3_pass_fail'], axis=1) # exclude independent variables
y = df1['G3_pass_fail']  # only include dependent variable data

In [13]:
df1_dummy = pd.get_dummies(data=df1, drop_first=True)
df1_dummy.head(5)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_home,reason_other,reason_reputation,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,G3_pass_fail_pass,G3_five_level_II,G3_five_level_III,G3_five_level_IV,G3_five_level_V
0,18,4,4,2,2,0,4,3,4,1,1,3,4,0,11,11,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,0
1,17,1,1,1,2,0,5,3,3,1,1,3,2,9,11,11,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0
2,15,1,1,1,2,0,4,3,2,2,3,3,6,12,13,12,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,0,0
3,15,4,2,1,3,0,3,2,2,1,1,5,0,14,14,14,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0
4,16,3,3,1,2,0,4,3,2,1,2,5,0,11,13,13,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0


In [12]:
X = df1_dummy.drop(['G3', 'G3_pass_fail_pass'], axis=1)
y = df1_dummy['G3_pass_fail_pass']

In [14]:
#split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# numerical features
num_cols = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
            'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Standardization
X_train_stand = X_train.copy() # copy of datasets
X_test_stand = X_test.copy()

for i in num_cols:
    scale = StandardScaler().fit(X_train_stand[[i]])
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

    
# Normalization
X_train_norm = X_train.copy() # copy of datasets
X_test_norm = X_test.copy()

for i in num_cols:
    norm = MinMaxScaler().fit(X_train_norm[[i]])
    X_train_norm[i] = norm.transform(X_train_norm[[i]])
    X_test_norm[i] = norm.transform(X_test_norm[[i]])

In [15]:
X_train.shape

(486, 41)

In [28]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
param_grid = {"n_neighbors":[10,20,50,100,200,300]}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train) 


#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))


ValueError: could not convert string to float: 'MS'

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 20)
knn.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(knn.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(knn.score(X_test, y_test)))

knn.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(knn.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(knn.score(X_test_stand, y_test)))

knn.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(knn.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(knn.score(X_test_norm, y_test)))

In [ ]:
# Logistic Regression
logreg = LogisticRegression().fit(X_train, y_train)
print("Training set RAW: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score RAW: {:.3f}".format(logreg.score(X_test, y_test)))

logreg = LogisticRegression().fit(X_train_stand, y_train)
print("Training set Standardized: {:.3f}".format(logreg.score(X_train_stand, y_train)))
print("Test set score Standardized: {:.3f}".format(logreg.score(X_test_stand, y_test)))

logreg = LogisticRegression().fit(X_train_norm, y_train)
print("Training set Normalized: {:.3f}".format(logreg.score(X_train_norm, y_train)))
print("Test set score Normalized: {:.3f}".format(logreg.score(X_test_norm, y_test)))

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
param_grid = {"splitter":['best', 'random'], "max_depth":[5,10,15,20], "min_samples_split": [5,10,15,20], "max_features":['auto', 'sqrt', 'log2']}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
tree = DecisionTreeClassifier(max_depth=5, max_features='auto', min_samples_split=10)
tree.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(tree.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(tree.score(X_test, y_test)))

tree.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(tree.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(tree.score(X_test_stand, y_test)))

tree.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(tree.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(tree.score(X_test_norm, y_test)))

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
param_grid = {'bootstrap': [True, False],
        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'max_features': ['auto', 'sqrt','log2'],
        'min_samples_split': [5,10,15,20],
        'n_estimators': [10, 20, 40, 60, 80, 100]}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
rf = RandomForestClassifier(bootstrap=False, max_depth=90, max_features='sqrt', min_samples_split=15, n_estimators=60)
rf.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(rf.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(rf.score(X_test, y_test)))

rf.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(rf.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(rf.score(X_test_stand, y_test)))

rf.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(rf.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(rf.score(X_test_norm, y_test)))

In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'learning_rate': [0.001, 0.01, 0.1, 1],
        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'max_features': ['auto', 'sqrt','log2'],
        'min_samples_split': [50, 80, 100, 150],
        'n_estimators': [10, 20, 50, 100, 200]}
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
gb = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20)
gb.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(gb.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(gb.score(X_test, y_test)))

gb.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(gb.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(gb.score(X_test_stand, y_test)))

gb.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(gb.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(gb.score(X_test_norm, y_test)))

In [ ]:
# SVM
from sklearn.svm import SVC
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf', 'sigmoid','poly'],'degree': [2,3,5,7]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
# SVM scaled
from sklearn.svm import SVC
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf', 'sigmoid','poly'],'degree': [2,3,5,7]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train_stand, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test_stand, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
svc = SVC(C=100, degree=7, gamma=0.001, kernel='poly')
svc.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(svc.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(svc.score(X_test, y_test)))

svc.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_test_stand, y_test)))

svc.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_test_norm, y_test)))

print("\n")
svc = SVC(C=1000, degree=2, gamma=1, kernel='linear')
svc.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(svc.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(svc.score(X_test, y_test)))

svc.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_test_stand, y_test)))

svc.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_test_norm, y_test)))

In [ ]:
# Feature Selection
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)

mask = select.get_support()
# visualize the mask. black is True, white is False
plt.matshow(mask.reshape(1, -1), cmap='gray_r')
plt.xlabel("Sample index")
plt.yticks(())

#transform then apply LogisticRegression
X_test_l1 = select.transform(X_test)
score = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20).fit(X_train_l1, y_train).score(X_test_l1, y_test)
print("Test score: {:.3f}".format(score))


In [ ]:
# Evaluation method
from sklearn.metrics import classification_report, roc_curve, auc
gb = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20).fit(X_train, y_train)
print(classification_report(y_test, gb.predict(X_test)))

from sklearn.metrics import roc_auc_score
gb_auc = roc_auc_score(y_test, gb.decision_function(X_test))
print("AUC for GB: {:.3f}".format(gb_auc))

# Scenario 2 -  Classification with five levels (from I - excellent to V - insufficient)

In [17]:
df2 = df

In [18]:
# create five level categorical variable for classification with five levels (from I - excellent to V - insufficient); and
conditions = [
    (df['G3'] <= 4),
    (df['G3'] > 4) & (df['G3'] <=8),
    (df['G3'] > 8) & (df['G3'] <=12),
    (df['G3'] > 12) & (df['G3'] <=16),
    (df['G3'] > 16)
    ]

# create a list of the values we want to assign for each condition
values = ['I', 'II', 'III', 'IV', 'V' ]

# create a new column and use np.select to assign values to it using our lists as arguments
df2['G3_five_level'] = np.select(conditions, values)

In [31]:
df2_dummy = pd.get_dummies(data=df2, drop_first=True)
df2_dummy.head(5)

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_home,reason_other,reason_reputation,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,G3_pass_fail_pass,G3_five_level_II,G3_five_level_III,G3_five_level_IV,G3_five_level_V
0,18,4,4,2,2,0,4,3,4,1,1,3,4,0,11,11,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,0
1,17,1,1,1,2,0,5,3,3,1,1,3,2,9,11,11,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0
2,15,1,1,1,2,0,4,3,2,2,3,3,6,12,13,12,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,0,0
3,15,4,2,1,3,0,3,2,2,1,1,5,0,14,14,14,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0
4,16,3,3,1,2,0,4,3,2,1,2,5,0,11,13,13,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0


In [32]:
X = df2.drop(['G3', 'G3_five_level'], axis=1) # exclude independent variables
y = df2['G3_five_level']  # only include dependent variable data


In [33]:
X = df2_dummy.drop(['G3', 'G3_pass_fail_pass'], axis=1)
y = df2_dummy['G3_pass_fail_pass']

In [25]:
df2.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pass_fail,G3_five_level
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11,pass,III
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11,pass,III
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12,pass,III
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14,pass,IV
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13,pass,IV


In [34]:
#split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# numerical features
num_cols = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
            'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Standardization
X_train_stand = X_train.copy() # copy of datasets
X_test_stand = X_test.copy()

for i in num_cols:
    scale = StandardScaler().fit(X_train_stand[[i]])
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

    
# Normalization
X_train_norm = X_train.copy() # copy of datasets
X_test_norm = X_test.copy()

for i in num_cols:
    norm = MinMaxScaler().fit(X_train_norm[[i]])
    X_train_norm[i] = norm.transform(X_train_norm[[i]])
    X_test_norm[i] = norm.transform(X_test_norm[[i]])

In [35]:
#Data visualization
#pd.plotting.scatter_matrix(df2, alpha=0.2, figsize=(20,20))

In [36]:
# KNN GridSearchCV
param_grid = {"n_neighbors":range(2,10)}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train) 


#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))


Test set score: 0.90
Best parameters: {'n_neighbors': 7}
Best cross-validation score: 0.87
Best estimator:
KNeighborsClassifier(n_neighbors=7)


In [40]:
# KNN
knn = KNeighborsClassifier(n_neighbors = 20)
knn.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(knn.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(knn.score(X_test, y_test)))

knn.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(knn.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(knn.score(X_test_stand, y_test)))

knn.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(knn.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(knn.score(X_test_norm, y_test)))

Train set accuracy RAW: 0.90
Test set accuracy RAW: 0.88
Test set accuracy Standardized: 0.85
Test set accuracy Standardized: 0.83
Test set accuracy Normalized: 0.82
Test set accuracy Normalized: 0.82


In [41]:
# Logistic Regression
logreg = LogisticRegression().fit(X_train, y_train)
print("Training set RAW: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score RAW: {:.3f}".format(logreg.score(X_test, y_test)))

logreg = LogisticRegression().fit(X_train_stand, y_train)
print("Training set Standardized: {:.3f}".format(logreg.score(X_train_stand, y_train)))
print("Test set score Standardized: {:.3f}".format(logreg.score(X_test_stand, y_test)))

logreg = LogisticRegression().fit(X_train_norm, y_train)
print("Training set Normalized: {:.3f}".format(logreg.score(X_train_norm, y_train)))
print("Test set score Normalized: {:.3f}".format(logreg.score(X_test_norm, y_test)))

Training set RAW: 0.944
Test set score RAW: 0.890
Training set Standardized: 0.934
Test set score Standardized: 0.890
Training set Normalized: 0.897
Test set score Normalized: 0.847


In [42]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
param_grid = {"splitter":['best', 'random'], "max_depth":[5,10,15,20], "min_samples_split": [5,10,15,20], "max_features":['auto', 'sqrt', 'log2']}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.91
Best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 15, 'splitter': 'best'}
Best cross-validation score: 0.89
Best estimator:
DecisionTreeClassifier(max_depth=5, max_features='sqrt', min_samples_split=15)


In [43]:
tree = DecisionTreeClassifier(max_depth=5, max_features='auto', min_samples_split=10)
tree.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(tree.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(tree.score(X_test, y_test)))

tree.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(tree.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(tree.score(X_test_stand, y_test)))

tree.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(tree.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(tree.score(X_test_norm, y_test)))

Train set accuracy RAW: 0.90
Test set accuracy RAW: 0.90
Test set accuracy Standardized: 0.91
Test set accuracy Standardized: 0.90
Test set accuracy Normalized: 0.91
Test set accuracy Normalized: 0.87


In [44]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
param_grid = {'bootstrap': [True, False],
        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'max_features': ['auto', 'sqrt','log2'],
        'min_samples_split': [5,10,15,20],
        'n_estimators': [10, 20, 40, 60, 80, 100]}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.91
Best parameters: {'bootstrap': True, 'max_depth': 100, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation score: 0.92
Best estimator:
RandomForestClassifier(max_depth=100, min_samples_split=5)


In [45]:
rf = RandomForestClassifier(bootstrap=False, max_depth=90, max_features='sqrt', min_samples_split=15, n_estimators=60)
rf.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(rf.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(rf.score(X_test, y_test)))

rf.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(rf.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(rf.score(X_test_stand, y_test)))

rf.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(rf.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(rf.score(X_test_norm, y_test)))

Train set accuracy RAW: 0.96
Test set accuracy RAW: 0.91
Test set accuracy Standardized: 0.97
Test set accuracy Standardized: 0.93
Test set accuracy Normalized: 0.98
Test set accuracy Normalized: 0.91


In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'learning_rate': [0.001, 0.01, 0.1, 1],
        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'max_features': ['auto', 'sqrt','log2'],
        'min_samples_split': [50, 80, 100, 150],
        'n_estimators': [10, 20, 50, 100, 200]}
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
gb = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20)
gb.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(gb.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(gb.score(X_test, y_test)))

gb.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(gb.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(gb.score(X_test_stand, y_test)))

gb.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(gb.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(gb.score(X_test_norm, y_test)))

In [ ]:
# SVM
from sklearn.svm import SVC
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf', 'sigmoid','poly'],'degree': [2,3,5,7]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
# SVM scaled
from sklearn.svm import SVC
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf', 'sigmoid','poly'],'degree': [2,3,5,7]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train_stand, y_train)

#score on test set    
print("Test set score: {:.2f}".format(grid_search.score(X_test_stand, y_test)))

#best parameters that were found
print("Best parameters: {}".format(grid_search.best_params_))
#best cross-validation accuracy (the mean accuracy over the different splits for this parameter setting)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
#access the actual model that was found (look at coefficients or feature importances)
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
svc = SVC(C=100, degree=7, gamma=0.001, kernel='poly')
svc.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(svc.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(svc.score(X_test, y_test)))

svc.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_test_stand, y_test)))

svc.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_test_norm, y_test)))

print("\n")
svc = SVC(C=1000, degree=2, gamma=1, kernel='linear')
svc.fit(X_train, y_train)
print("Train set accuracy RAW: {:.2f}".format(svc.score(X_train, y_train)))
print("Test set accuracy RAW: {:.2f}".format(svc.score(X_test, y_test)))

svc.fit(X_train_stand, y_train)
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_train_stand, y_train)))
print("Test set accuracy Standardized: {:.2f}".format(svc.score(X_test_stand, y_test)))

svc.fit(X_train_norm, y_train)
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_train_norm, y_train)))
print("Test set accuracy Normalized: {:.2f}".format(svc.score(X_test_norm, y_test)))

In [ ]:
# Feature Selection
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)

mask = select.get_support()
# visualize the mask. black is True, white is False
plt.matshow(mask.reshape(1, -1), cmap='gray_r')
plt.xlabel("Sample index")
plt.yticks(())

#transform then apply LogisticRegression
X_test_l1 = select.transform(X_test)
score = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20).fit(X_train_l1, y_train).score(X_test_l1, y_test)
print("Test score: {:.3f}".format(score))

In [ ]:
# Evaluation method
from sklearn.metrics import classification_report, roc_curve, auc
gb = GradientBoostingClassifier(max_depth=10, max_features='auto',
                           min_samples_split=150, n_estimators=20).fit(X_train, y_train)
print(classification_report(y_test, gb.predict(X_test)))

from sklearn.metrics import roc_auc_score
gb_auc = roc_auc_score(y_test, gb.decision_function(X_test))
print("AUC for GB: {:.3f}".format(gb_auc))

# Scenario 3 - Regression, with a numeric output that ranges between 0 and 20.

In [ ]:
df3 = df.copy()

In [ ]:
df3.head(5)

In [ ]:
df3.columns

In [ ]:
# credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 
# One of the best notebooks on getting started with a ML problem.

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# let's take a look at missingness
df3_missing = missing_values_table(df3)
df3_missing

### Exploratory Data Analysis

In [ ]:
df3.dtypes

In [ ]:
sns.countplot(x='school', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='sex', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='famsize', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='activities', data=df3, palette='hls')
plt.show()

In [ ]:
#creating a countplot to show the distribution of G3
sns.countplot(x='G3', data=df3, palette='hls')
plt.show()

### Feature Transformation

A rule of thumb, skewness can be interpreted like this:

* Fairly symmetrical = -0.5 to 0.5
* Moderately skewed= -0.5 to -1.0 and 0.5 to 1.0
* Highly skewed = <-1.0 and >1.0

From the table above, it seems like **traveltime**, **failures**, **famrel**, **Dalc**, and **absences** are highly skewed and **studytime**, **Walc**, and **G3** are moderately skewed. Let's take a look at their histogram

In [ ]:
df3.agg(['skew', 'kurtosis']).transpose()

In [ ]:
sns.countplot(x='traveltime', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='failures', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='famrel', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='Dalc', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='absences', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='studytime', data=df3, palette='hls')
plt.show()

In [ ]:
sns.countplot(x='Walc', data=df3, palette='hls')
plt.show()

In [ ]:
df3_dummy = pd.get_dummies(data=df3, drop_first=True) # Do not use fit_intercept = False if you have removed 1 column after dummy encoding

In [ ]:
X = df3_dummy.drop(['G3'], axis=1) # exclude independent variables and variables that have been transformed
y = df3_dummy['G3']  # only include dependent variable data

In [ ]:
df3.select_dtypes("int").columns

In [ ]:
# Raw
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# numerical features
num_cols = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
            'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Standardization
X_train_stand = X_train.copy() # copy of datasets
X_test_stand = X_test.copy()

for i in num_cols:
    scale = StandardScaler().fit(X_train_stand[[i]])
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

    
# Normalization
X_train_norm = X_train.copy() # copy of datasets
X_test_norm = X_test.copy()

for i in num_cols:
    norm = MinMaxScaler().fit(X_train_norm[[i]])
    X_train_norm[i] = norm.transform(X_train_norm[[i]])
    X_test_norm[i] = norm.transform(X_test_norm[[i]])

In [ ]:
# Linear Regression
model = LinearRegression()

grid={}
model_cv=GridSearchCV(model,grid,cv=5, scoring='r2')
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Ridge
from sklearn.linear_model import Ridge
model = Ridge()

grid={"alpha": [0.01, 0.1, 1,10], "solver":['svd', 'cholesky', 'lsqr', 'sag'], "fit_intercept":[False], "normalize":[True, False]}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = Ridge(alpha=10, fit_intercept=False, normalize= True, solver='lsqr')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()

grid={"n_neighbors":range(2,10)}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = KNeighborsRegressor(n_neighbors=6)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Lasso
from sklearn.linear_model import Lasso
model = Lasso()

grid={'alpha':[0.01, 0.1, 1.0, 10]}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = Lasso(alpha=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()

grid={"min_samples_split": range(2,10), "splitter":['best', 'random'], "max_depth":range(1,10), "max_features":['auto', 'sqrt', 'log2']}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = DecisionTreeRegressor(max_depth=3, max_features='auto', min_samples_split=2, splitter='best')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=0)

grid={'bootstrap': [True, False],
      'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
      'max_features': ['auto', 'sqrt'],
      'min_samples_leaf': [1, 2, 4],
      'min_samples_split': [2, 5, 10],
      'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Gradient Boosted Trees
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=0)

grid={}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = GradientBoostingRegressor(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))

In [ ]:
# Support Vector Machine
from sklearn.svm import SVR
model = SVR()

grid={}
model_cv=GridSearchCV(model,grid,cv=5)
model_cv.fit(X_train,y_train)
y_pred = model_cv.predict(X_test)
print("Best parameters: ", model_cv.best_params_)

model = SVR()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('R2 for Raw:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)
print('R2 for Normalized:', metrics.r2_score(y_test, y_pred))

model.fit(X_train_stand, y_train)
y_pred = model.predict(X_test_stand)
print('R2 for Standardized:', metrics.r2_score(y_test, y_pred))